# What

This has been made because the issues of https://github.com/1jamesthompson1/TAIC-report-summary/issues/236. Has brought to attention that I need to easily be able to look at the logs.

## Modules

In [ ]:
import pandas as pd
import dotenv
from azure.data.tables import TableServiceClient
import os
from io import StringIO

dotenv.load_dotenv(override=True)

# Get the tables

In [ ]:
connection_string = f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT_NAME')};AccountKey={os.getenv('AZURE_STORAGE_ACCOUNT_KEY')};EndpointSuffix=core.windows.net"
client = TableServiceClient.from_connection_string(conn_str=connection_string)
search_table_client = client.get_table_client(table_name="searchlogs")
results_table_client = client.get_table_client(table_name="resultslogs")
errors_table_client = client.get_table_client(table_name="errorlogs")

In [ ]:
entities = search_table_client.list_entities()
data = [entity for entity in entities]
search_logs = pd.DataFrame(data)
search_logs['start_time'] = pd.to_datetime(search_logs['start_time'], unit='s', utc=True).map(lambda x: x.tz_convert('NZ'))
search_logs.sort_values(by='start_time', ascending=False, inplace=True)

search_logs

In [ ]:
results_logs = pd.DataFrame(results_table_client.list_entities())

results_logs['search_results'] = results_logs['search_results'].map(lambda x: pd.read_json(StringIO(x)))

results_logs

In [ ]:
error_logs = pd.DataFrame(errors_table_client.list_entities())

error_logs

# Merge logs together

In [ ]:
search_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)
results_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)
error_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)

merged_logs = search_logs.join([results_logs, error_logs])

merged_logs = merged_logs[merged_logs.index.get_level_values('PartitionKey') != 'Test User']

merged_logs = merged_logs[merged_logs['start_time'] >= pd.Timestamp('2024-07-29 00:00:00', tz='NZ')]
merged_logs

In [ ]:
merged_logs.index.get_level_values(0).value_counts()